# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart_failure_automl2'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
Environment.from_conda_specification(name='sklearn-env2', file_path='conda_env_v_1_0_0.yml')

{
    "assetId": null,
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230509.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "sklearn-env2",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
        

In [4]:
compute_name = "optCompute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
import pandas as pd
from azureml.train.automl import AutoMLConfig

dataset = Dataset.get_by_name(ws, name='heart_rate_failure_prediction')  



automl_settings = {
    "experiment_timeout_minutes" : 25,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=trainCluster,
                             training_data = dataset,
                             label_column_name = "HeartDisease",
                             enable_onnx_compatible_models=False,
                             **automl_settings
                            )  




  







In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart_failure_automl2,AutoML_0f01a500-d600-4d69-9a74-ff800fcc162a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: heart_failure_automl2,
Id: AutoML_0f01a500-d600-4d69-9a74-ff800fcc162a_35,
Type: azureml.scriptrun,
Status: Completed)
f1_score_macro 0.8790130235240335
f1_score_weighted 0.8809401752046437
average_precision_score_weighted 0.9351455312641871
log_loss 0.4083195044027777
recall_score_macro 0.8771476393034032
matthews_correlation 0.7599270761965813
recall_score_micro 0.8812960323117129
average_precision_score_micro 0.9356143463969839
weighted_accuracy 0.8852195968920438
accuracy 0.8812960323117129
AUC_micro 0.9367265568636622
f1_score_micro 0.8812960323117129
precision_score_macro 0.8828157705702122
precision_score_micro 0.8812960323117129
average_precision_score_macro 0.9341435824374289
norm_macro_recall 0.7542952786068065
AUC_macro 0.9345037896796144
precision_score_weighted 0.8824615353449212
AUC_weighted 0.9345037896796142
recall_score_weighted 0.8812960323117129
balanced_accuracy 0.8771476393034032
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_0f01a500-d

In [11]:
#TODO: Save the best model

from azureml.core import Model # Used to get model information
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            print({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            print(step[1].get_params())
            print()

print_model(fitted_model)

myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)


# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

datatransformer
{'task': 'classification', 'is_onnx_compatible': False, 'enable_feature_sweeping': True, 'enable_dnn': False, 'force_text_dnn': False, 'feature_sweeping_timeout': 86400, 'featurization_config': None, 'is_cross_validation': True, 'feature_sweeping_config': {}, 'observer': None, 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook23/code/Users/odl_user_251683/starter_file'}

prefittedsoftvotingclassifier
{'estimators': ['18', '17', '30', '15', '14', '25', '21', '13', '3'], 'weights': [0.06666666666666667, 0.2, 0.13333333333333333, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.2]}

18 - standardscalerwrapper
{'module_name': 'sklearn.preprocessing._data', 'class_name': 'StandardScaler', 'copy': True, 'with_mean': False, 'with_std': False}

18 - xgboostclassifier
{'objective': 'reg:logistic', 'use_label_encoder': True, 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_by

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-05 01:57:13+00:00 Creating Container Registry if not exists.
2024-02-05 01:57:14+00:00 Use the existing image.
2024-02-05 01:57:14+00:00 Submitting deployment to compute.
2024-02-05 01:57:19+00:00 Checking the status of deployment myservice..
2024-02-05 01:59:55+00:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://d4af88e1-687a-47da-ab8f-bf6e8d85ba14.westeurope.azurecontainer.io/score
http://d4af88e1-687a-47da-ab8f-bf6e8d85ba14.westeurope.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
import json

#Import test data
data = dataset.to_pandas_dataframe().dropna()
test_df = data.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('HeartDisease')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)

{"data": [{"Age": 54, "Sex": "M", "ChestPainType": "ATA", "RestingBP": 132, "Cholesterol": 182, "FastingBS": 0, "RestingECG": "ST", "MaxHR": 141, "ExerciseAngina": false, "Oldpeak": 0.1, "ST_Slope": "Up"}, {"Age": 41, "Sex": "M", "ChestPainType": "ATA", "RestingBP": 110, "Cholesterol": 235, "FastingBS": 0, "RestingECG": "Normal", "MaxHR": 153, "ExerciseAngina": false, "Oldpeak": 0.0, "ST_Slope": "Up"}, {"Age": 52, "Sex": "M", "ChestPainType": "ASY", "RestingBP": 125, "Cholesterol": 212, "FastingBS": 0, "RestingECG": "Normal", "MaxHR": 168, "ExerciseAngina": false, "Oldpeak": 1.0, "ST_Slope": "Up"}, {"Age": 33, "Sex": "F", "ChestPainType": "ASY", "RestingBP": 100, "Cholesterol": 246, "FastingBS": 0, "RestingECG": "Normal", "MaxHR": 150, "ExerciseAngina": true, "Oldpeak": 1.0, "ST_Slope": "Flat"}, {"Age": 59, "Sex": "F", "ChestPainType": "ASY", "RestingBP": 130, "Cholesterol": 338, "FastingBS": 1, "RestingECG": "ST", "MaxHR": 130, "ExerciseAngina": true, "Oldpeak": 1.5, "ST_Slope": "Flat

TODO: In the cell below, print the logs of the web service and delete the service

In [18]:
# Print results from the inference
print(response.text)

# Print original labels
print(label_df)

print(service.get_logs())

#service.delete()

"{\"result\": [0, 0, 0, 1, 1]}"
561    0
726    0
709    1
115    1
117    1
Name: HeartDisease, dtype: int64


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
